In [1]:
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_color_codes("pastel")
plt.rcParams["figure.figsize"] = [20, 8]
from pyspark.sql.functions import udf
from pyspark.ml.linalg import DenseVector, SparseVector, Vectors, VectorUDT

In [2]:
content_cols = '''
ID - Unique identifier for the record.
Case Number - The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.
Date - Date when the incident occurred. this is sometimes a best estimate.
Block - The partially redacted address where the incident occurred, placing it on the same block as the actual address.
IUCR - The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.
Primary Type - The primary description of the IUCR code.
Description - The secondary description of the IUCR code, a subcategory of the primary description.
Location Description - Description of the location where the incident occurred.
Arrest - Indicates whether an arrest was made.
Domestic - Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.
Beat - Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74.
District - Indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r.
Ward - The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76.
Community Area - Indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6.
FBI Code - Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html.
X Coordinate - The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.
Y Coordinate - The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.
Year - Year the incident occurred.
Updated On - Date and time the record was last updated.
Latitude - The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.
Longitude - The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.
Location - The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.'''

In [3]:
def entry_dic(line):
    """
    Convert a header - description line into a single dictionary that holds the original header as 'title', a corresponding field name as 'header', and a description.
    """
    pair = line.split(' - ')
    return {'title': pair[0], 'description': pair[1], 'header': pair[0].lower().replace(' ', '_')}

In [4]:
header_dics = list(map(entry_dic, list(filter(lambda l: l != '', content_cols.split('\n')))))

In [5]:
header_dics[:2]

Out[6]: [{'title': 'ID',
 'description': 'Unique identifier for the record.',
 'header': 'id'},
 {'title': 'Case Number',
 'description': 'The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.',
 'header': 'case_number'}]

In [6]:
df = spark.read.csv("/FileStore/tables/Crimes___2001_to_present-9be1b.csv" , inferSchema=True, header=True)

In [7]:
# Caching this data frame as it's going to be read over and over again
df = df.cache()

In [8]:
df.printSchema()

root
-- ID: integer (nullable = true)
-- Case Number: string (nullable = true)
-- Date: string (nullable = true)
-- Block: string (nullable = true)
-- IUCR: string (nullable = true)
-- Primary Type: string (nullable = true)
-- Description: string (nullable = true)
-- Location Description: string (nullable = true)
-- Arrest: boolean (nullable = true)
-- Domestic: boolean (nullable = true)
-- Beat: integer (nullable = true)
-- District: integer (nullable = true)
-- Ward: integer (nullable = true)
-- Community Area: integer (nullable = true)
-- FBI Code: string (nullable = true)
-- X Coordinate: integer (nullable = true)
-- Y Coordinate: integer (nullable = true)
-- Year: integer (nullable = true)
-- Updated On: string (nullable = true)
-- Latitude: double (nullable = true)
-- Longitude: double (nullable = true)
-- Location: string (nullable = true)

In [9]:
for h in header_dics:
    df = df.withColumnRenamed(h['title'], h['header'])

In [10]:
df = df.rdd.filter(lambda rec: rec.arrest.find('Location Description') < 0).toDF().cache()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4442033637449617> in <module> 
 ----> 1 df = df . rdd . filter ( lambda rec : rec . arrest . find ( 'Location Description' ) < 0 ) . toDF ( ) . cache ( ) 

 /databricks/spark/python/pyspark/sql/session.py in toDF (self, schema, sampleRatio) 
 58 [ Row ( name = u'Alice' , age = 1 ) ] 
 59 """
 ---> 60 return sparkSession . createDataFrame ( self , schema , sampleRatio ) 
 61 
 62 RDD . toDF = toDF

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 813 else : 
 814 if isinstance ( data , RDD ) : 
 --> 815 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 816 else : 
 817 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromRDD (self, rdd, schema, samplingRatio) 
 397 """
 398 if schema is None or isinstance ( schema , ( list , tuple ) ) : 
 --> 399 struct = self . _inferSchema ( rdd , samplingRatio , names = schema ) 
 400 converter = _create_converter ( struct ) 
 401 rdd = rdd . map ( converter ) 

 /databricks/spark/python/pyspark/sql/session.py in _inferSchema (self, rdd, samplingRatio, names) 
 368 : return : : class : ` pyspark . sql . types . StructType ` 
 369 """
 --> 370 first = rdd . first ( ) 
 371 if not first : 
 372 raise ValueError("The first row in RDD is empty, "

 /databricks/spark/python/pyspark/rdd.py in first (self) 
 1417 ValueError : RDD is empty
 1418 """
 -> 1419 rs = self . take ( 1 ) 
 1420 if rs : 
 1421 return rs [ 0 ] 

 /databricks/spark/python/pyspark/rdd.py in take (self, num) 
 1399 
 1400 p = range ( partsScanned , min ( partsScanned + numPartsToTry , totalParts ) ) 
 -> 1401 res = self . context . runJob ( self , takeUpToNumLeft , p ) 
 1402 
 1403 items += res

 /databricks/spark/python/pyspark/context.py in runJob (self, rdd, partitionFunc, partitions, allowLocal) 
 1176 finally : 
 1177 os . remove ( filename ) 
 -> 1178 sock_info = self . _jvm . PythonRDD . runJob ( self . _jsc . sc ( ) , mappedRDD . _jrdd , partitions ) 
 1179 return list ( _load_from_socket ( sock_info , mappedRDD . _jrdd_deserializer ) ) 
 1180 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 14, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 480, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 472, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 508, in dump_stream
 vs = list(itertools.islice(iterator, batch))
 File "/databricks/spark/python/pyspark/rdd.py", line 1395, in takeUpToNumLeft
 yield next(iterator)
 File "/databricks/spark/python/pyspark/

In [ ]:
df.show(n=3)

In [12]:
# crime types
crime_type_groups = df.groupBy('primary_type').count()

In [13]:
crime_type_counts = crime_type_groups.orderBy('count', ascending=False)

In [14]:
df.columns

Out[15]: ['id',
 'case_number',
 'date',
 'block',
 'iucr',
 'primary_type',
 'description',
 'location_description',
 'arrest',
 'domestic',
 'beat',
 'district',
 'ward',
 'community_area',
 'fbi_code',
 'x_coordinate',
 'y_coordinate',
 'year',
 'updated_on',
 'latitude',
 'longitude',
 'location']

In [15]:
#Let's see the schema of the data frame:
df.printSchema()

root
-- id: integer (nullable = true)
-- case_number: string (nullable = true)
-- date: string (nullable = true)
-- block: string (nullable = true)
-- iucr: string (nullable = true)
-- primary_type: string (nullable = true)
-- description: string (nullable = true)
-- location_description: string (nullable = true)
-- arrest: boolean (nullable = true)
-- domestic: boolean (nullable = true)
-- beat: integer (nullable = true)
-- district: integer (nullable = true)
-- ward: integer (nullable = true)
-- community_area: integer (nullable = true)
-- fbi_code: string (nullable = true)
-- x_coordinate: integer (nullable = true)
-- y_coordinate: integer (nullable = true)
-- year: integer (nullable = true)
-- updated_on: string (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- location: string (nullable = true)

In [16]:
crime_type_counts.show(truncate=False)

+--------------------------------+-------+
primary_type |count |
+--------------------------------+-------+
THEFT |1494719|
BATTERY |1291953|
CRIMINAL DAMAGE |803703 |
NARCOTICS |728789 |
ASSAULT |443370 |
OTHER OFFENSE |439195 |
BURGLARY |400016 |
MOTOR VEHICLE THEFT |325331 |
DECEPTIVE PRACTICE |287123 |
ROBBERY |265757 |
CRIMINAL TRESPASS |201692 |
WEAPONS VIOLATION |78274 |
PROSTITUTION |69136 |
PUBLIC PEACE VIOLATION |49620 |
OFFENSE INVOLVING CHILDREN |48486 |
CRIM SEXUAL ASSAULT |29336 |
SEX OFFENSE |26785 |
INTERFERENCE WITH PUBLIC OFFICER|16983 |
GAMBLING |14571 |
LIQUOR LAW VIOLATION |14331 |
+--------------------------------+-------+
only showing top 20 rows

In [17]:
counts_pddf = pd.DataFrame(crime_type_counts.rdd.map(lambda l: l.asDict()).collect())

In [18]:
counts_pddf.head(10)

,count,primary_type
0,1494719,THEFT
1,1291953,BATTERY
2,803703,CRIMINAL DAMAGE
3,728789,NARCOTICS
4,443370,ASSAULT
5,439195,OTHER OFFENSE
6,400016,BURGLARY
7,325331,MOTOR VEHICLE THEFT
8,287123,DECEPTIVE PRACTICE
9,265757,ROBBERY


In [19]:
plt.rcParams["figure.figsize"] = [20, 8]

sns.set(style="whitegrid")
sns.set_color_codes("dark")

#sns.despine(left=True, bottom=True)
type_graph = sns.barplot(x='count', y='primary_type', data=counts_pddf)
type_graph.set(ylabel="Primary Type", xlabel="Crimes Record Count")
plt.xlabel("Number of crimes", fontsize = 10)
plt.ylabel("Crime Location", fontsize = 10)
plt.title("Number of Crimes By Location", fontsize = 20)
plt.xticks(size = 10)
plt.yticks(size = 8)
display()

In [20]:
import datetime
from pyspark.sql.functions import *

In [21]:
df.select(min('date').alias('first_record_date'), max('date').alias('latest_record_date')).show(truncate=False)

+----------------------+----------------------+
first_record_date |latest_record_date |
+----------------------+----------------------+
01/01/2001 01:00:00 AM|12/31/2019 12:58:00 AM|
+----------------------+----------------------+

In [22]:
df = df.withColumn('date_time', to_timestamp('date', 'MM/dd/yyyy hh:mm:ss a'))\
       .withColumn('month', trunc('date_time', 'YYYY')) #adding a month column to be able to view stats on a monthly basis

In [23]:
df.select(['date','date_time', 'month'])\
  .show(n=2, truncate=False)

+----------------------+-------------------+----------+
date |date_time |month |
+----------------------+-------------------+----------+
01/01/2001 11:00:00 AM|2001-01-01 11:00:00|2001-01-01|
10/08/2017 03:00:00 AM|2017-10-08 03:00:00|2017-01-01|
+----------------------+-------------------+----------+
only showing top 2 rows

In [24]:
# crime types and arrest over years:
type_arrest_date = df.groupBy(['arrest', 'month'])\
                     .count()\
                     .orderBy(['month', 'count'], ascending=[True, False])
print()
type_arrest_date.show(3, truncate=False)

+------+----------+------+
arrest|month |count |
+------+----------+------+
false |2001-01-01|343858|
true |2001-01-01|141905|
false |2002-01-01|345207|
+------+----------+------+
only showing top 3 rows

In [25]:
# A small detour to learn/see how datetime works
import datetime

In [26]:
datetime.datetime.now()
datetime.datetime.strftime(datetime.datetime.now(), '%H')

Out[27]: '13'

In [27]:

# A pandas data frame of the collected dictionary version of the date-grouped DF above
type_arrest_pddf = pd.DataFrame(type_arrest_date.rdd.map(lambda l: l.asDict()).collect())

In [28]:
type_arrest_pddf['yearpd'] = type_arrest_pddf['month'].apply(lambda dt: datetime.datetime.strftime(pd.Timestamp(dt), '%Y'))

In [29]:
type_arrest_pddf['arrest'] = type_arrest_pddf['arrest'].apply(lambda l: l=='True')
type_arrest_pddf.head(5)

,arrest,count,month,yearpd
0,False,343858,2001-01-01,2001
1,False,141905,2001-01-01,2001
2,False,345207,2002-01-01,2002
3,False,141555,2002-01-01,2002
4,False,334376,2003-01-01,2003


In [30]:

# Data for plotting
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

t = type_arrest_pddf['count'] - 20 # np.arange(0.0, 2.0, 0.01)
s = type_arrest_pddf['month']

arrested = type_arrest_pddf[type_arrest_pddf['arrest'] == True]
not_arrested = type_arrest_pddf[type_arrest_pddf['arrest'] == False]

# Note that using plt.subplots below is equivalent to using
# fig = plt.figure() and then ax = fig.add_subplot(111)
fig, ax = plt.subplots()
ax.plot(arrested['month'], arrested['count'], label='Arrested')
ax.plot(not_arrested['month'], not_arrested['count'], label='Not Arrested')

ax.set(xlabel='Year - 2001-2017', ylabel='Total records',
       title='Year-on-year crime records')
ax.grid(b=True, which='both', axis='y')
ax.legend()
display()

In [31]:

# Extract the "hour" field from the date into a separate column called "hour"
df_hour = df.withColumn('hour', hour(df['date_time']))

In [32]:
# Derive a data frame with crime counts per hour of the day:
hourly_count = df_hour.groupBy(['primary_type', 'hour']).count().cache()
hourly_total_count = hourly_count.groupBy('hour').sum('count')

In [33]:
hourly_count_pddf = pd.DataFrame(hourly_total_count.select(hourly_total_count['hour'], hourly_total_count['sum(count)'].alias('count'))\
                                .rdd.map(lambda l: l.asDict())\
                                 .collect())

In [34]:
hourly_count_pddf = hourly_count_pddf.sort_values(by='hour')

In [35]:
fig, ax = plt.subplots()
ax.plot(hourly_count_pddf['hour'], hourly_count_pddf['count'], label='Hourly Count')

ax.set(xlabel='Hour of Day', ylabel='Total records',
       title='Overall hourly crime numbers')
ax.grid(b=True, which='both', axis='y')
ax.legend()
display()

In [36]:
# Number of types of location recorded in the dataset
df.select('location_description').distinct().count()

Out[37]: 181

In [37]:
###What are the top 10 places where crime occurred?
df.groupBy(['location_description']).count().orderBy('count', ascending=False).show(10)


+--------------------+-------+
location_description| count|
+--------------------+-------+
 STREET|1839454|
 RESIDENCE|1196676|
 APARTMENT| 740756|
 SIDEWALK| 689585|
 OTHER| 269815|
PARKING LOT/GARAG...| 202812|
 ALLEY| 156957|
SCHOOL, PUBLIC, B...| 146198|
 RESIDENCE-GARAGE| 135526|
 SMALL RETAIL STORE| 127841|
+--------------------+-------+
only showing top 10 rows

In [38]:
#Being on the street and at home
street_home_hour = location_hour.where((location_hour['location_description'] == 'STREET') | (location_hour['location_description'] == 'RESIDENCE'))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4442033637449645> in <module> 
 1 #Being on the street and at home 
 ----> 2 street_home_hour = location_hour . where ( ( location_hour [ 'location_description' ] == 'STREET' ) | ( location_hour [ 'location_description' ] == 'RESIDENCE' ) ) 

 NameError : name 'location_hour' is not defined

In [39]:
#a data frame with location descriptions and counts of recorded crimes, and hours...
street_home_hour_pddf = pd.DataFrame(street_home_hour.rdd.map(lambda row: row.asDict()).collect())
street_home_hour_pddf = street_home_hour_pddf.sort_values(by='hour')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4442033637449646> in <module> 
 1 #a data frame with location descriptions and counts of recorded crimes, and hours... 
 ----> 2 street_home_hour_pddf = pd . DataFrame ( street_home_hour . rdd . map ( lambda row : row . asDict ( ) ) . collect ( ) ) 
 3 street_home_hour_pddf = street_home_hour_pddf . sort_values ( by = 'hour' ) 

 NameError : name 'street_home_hour' is not defined

In [40]:
domestic_hour = pd.DataFrame(df_hour.groupBy(['domestic', 'hour']).count().orderBy('hour').rdd.map(lambda row: row.asDict()).collect())

In [41]:
dom = domestic_hour[domestic_hour['domestic'] == 'True']['count']
non_dom = domestic_hour[domestic_hour['domestic'] == 'False']['count']

either_dom = domestic_hour.groupby(by=['hour']).sum()['count']

dom_keys = domestic_hour[domestic_hour['domestic'] == 'False']['hour']

/databricks/python/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
 result = method(y)

In [42]:
figure, axes = plt.subplots()

axes.plot(dom_keys, either_dom, label='Total hourly count')
axes.plot(dom_keys, dom, label='Domestic crime count')
axes.plot(dom_keys, non_dom, label='Non-Domestic hourly count')

axes.legend()
axes.grid(which='b', b=True)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-4442033637449649> in <module> 
 1 figure , axes = plt . subplots ( ) 
 2 
 ----> 3 axes . plot ( dom_keys , either_dom , label = 'Total hourly count' ) 
 4 axes . plot ( dom_keys , dom , label = 'Domestic crime count' ) 
 5 axes . plot ( dom_keys , non_dom , label = 'Non-Domestic hourly count' ) 

 /databricks/python/lib/python3.7/site-packages/matplotlib/__init__.py in inner (ax, data, *args, **kwargs) 
 1808 "the Matplotlib list!)" % ( label_namer , func . __name__ ) , 
 1809 RuntimeWarning, stacklevel=2)
 -> 1810 return func ( ax , * args , ** kwargs ) 
 1811 
 1812 inner.__doc__ = _add_data_doc(inner.__doc__,

 /databricks/python/lib/python3.7/site-packages/matplotlib/axes/_axes.py in plot (self, scalex, scaley, *args, **kwargs) 
 1609 kwargs = cbook . normalize_kwargs ( kwargs , mlines . Line2D . _alias_map ) 
 1610 
 -> 1611 for line in self . _get_lines ( * args , ** kwargs ) : 
 1612 self . add_line ( line ) 
 1613 lines . append ( line ) 

 /databricks/python/lib/python3.7/site-packages/matplotlib/axes/_base.py in _grab_next_args (self, *args, **kwargs) 
 391 this += args [ 0 ] , 
 392 args = args [ 1 : ] 
 --> 393 yield from self . _plot_args ( this , kwargs ) 
 394 
 395 

 /databricks/python/lib/python3.7/site-packages/matplotlib/axes/_base.py in _plot_args (self, tup, kwargs) 
 368 x , y = index_of ( tup [ - 1 ] ) 
 369 
 --> 370 x , y = self . _xy_from_xy ( x , y ) 
 371 
 372 if self . command == 'plot' : 

 /databricks/python/lib/python3.7/site-packages/matplotlib/axes/_base.py in _xy_from_xy (self, x, y) 
 229 if x . shape [ 0 ] != y . shape [ 0 ] : 
 230 raise ValueError("x and y must have same first dimension, but "
 --> 231 "have shapes {} and {}".format(x.shape, y.shape))
 232 if x . ndim > 2 or y . ndim > 2 : 
 233 raise ValueError("x and y can be no greater than 2-D, but have "

 ValueError : x and y must have same first dimension, but have shapes (0,) and (24,)

In [43]:
df_dates = df_hour.withColumn('week_day', dayofweek(df_hour['date_time']))\
                 .withColumn('year_month', month(df_hour['date_time']))\
                 .withColumn('month_day', dayofmonth(df_hour['date_time']))\
                 .withColumn('date_number', datediff(df['date_time'], to_date(lit('2001-01-01'), format='yyyy-MM-dd')))\
                 .cache()

In [44]:
df_dates.select(['date', 'month', 'hour', 'week_day', 'year', 'year_month', 'month_day', 'date_number']).show(20, truncate=False)

+----------------------+----------+----+--------+----+----------+---------+-----------+
date |month |hour|week_day|year|year_month|month_day|date_number|
+----------------------+----------+----+--------+----+----------+---------+-----------+
01/01/2001 11:00:00 AM|2001-01-01|11 |2 |2001|1 |1 |0 |
10/08/2017 03:00:00 AM|2017-01-01|3 |1 |2017|10 |8 |6124 |
03/28/2017 02:00:00 PM|2017-01-01|14 |3 |2017|3 |28 |5930 |
09/09/2017 08:17:00 PM|2017-01-01|20 |7 |2017|9 |9 |6095 |
08/26/2017 10:00:00 AM|2017-01-01|10 |7 |2017|8 |26 |6081 |
02/10/2013 12:00:00 AM|2013-01-01|0 |1 |2013|2 |10 |4423 |
01/01/2015 12:01:00 AM|2015-01-01|0 |5 |2015|1 |1 |5113 |
01/01/2017 12:01:00 AM|2017-01-01|0 |1 |2017|1 |1 |5844 |
07/17/2017 10:10:00 AM|2017-01-01|10 |2 |2017|7 |17 |6041 |
12/28/2017 03:55:00 PM|2017-01-01|15 |5 |2017|12 |28 |6205 |
02/10/2017 12:00:00 PM|2017-01-01|12 |6 |2017|2 |10 |5884 |
11/22/2017 02:42:00 AM|2017-01-01|2 |4 |2017|11 |22 |6169 |
01/01/2012 09:00:00 AM|2012-01-01|9 |1 |2012|1 |1 |4017 |
07/29/2017 03:40:00 PM|2017-01-01|15 |7 |2017|7 |29 |6053 |
01/22/2017 12:01:00 AM|2017-01-01|0 |1 |2017|1 |22 |5865 |
10/14/2017 12:01:00 AM|2017-01-01|0 |7 |2017|10 |14 |6130 |
10/15/2014 03:00:00 PM|2014-01-01|15 |4 |2014|10 |15 |5035 |
10/17/2006 01:05:00 PM|2006-01-01|13 |3 |2006|10 |17 |2115 |
05/30/2015 12:00:00 AM|2015-01-01|0 |7 |2015|5 |30 |5262 |
05/30/2015 12:00:00 AM|2015-01-01|0 |7 |2015|5 |30 |5262 |
+----------------------+----------+----+--------+----+----------+---------+-----------+
only showing top 20 rows

In [45]:
week_day_crime_counts = df_dates.groupBy('week_day').count()


In [46]:
week_day_crime_counts_pddf = pd.DataFrame(week_day_crime_counts.orderBy('week_day').rdd.map(lambda e: e.asDict()).collect())

In [47]:

sns.barplot(data=week_day_crime_counts_pddf, x='week_day', y='count')
plt.title("Number of Crimes by Day", fontsize = 20)
display()

In [48]:
year_month_crime_counts = df_dates.groupBy('year_month').count()

In [49]:

year_month_crime_counts_pddf = pd.DataFrame(year_month_crime_counts.orderBy('year_month').rdd.map(lambda e: e.asDict()).collect())

In [50]:
year_month_crime_counts_pddf

,count,year_month
0,567501,1
1,485005,2
2,578050,3
3,580207,4
4,626841,5
5,623855,6
6,655559,7
7,648955,8
8,608065,9
9,614629,10


In [51]:
sns.barplot(data=year_month_crime_counts_pddf, y='count', x='year_month')
plt.title("Number of Crimes by Month", fontsize = 20)
display()

In [52]:
month_day_crime_counts = df_dates.groupBy('month_day').count()

In [53]:
month_day_crime_counts_pddf = pd.DataFrame(month_day_crime_counts.orderBy('month_day').rdd.map(lambda e: e.asDict()).collect())

In [54]:
#Top 10 worst days of the month
month_day_crime_counts_pddf.sort_values(by='count', ascending=False).head(10)

,count,month_day
0,278467,1
14,240222,15
19,235976,20
9,233566,10
16,232978,17
13,232622,14
15,232258,16
11,231454,12
17,231178,18
20,230949,21


In [55]:
month_day_crime_counts_pddf = month_day_crime_counts_pddf.sort_values(by='month_day', ascending=True)

In [56]:
df_dates_community_areas = df_dates.na.drop(subset=['community_area']).groupBy('community_area').count()

In [57]:

df_dates_community_areas.orderBy('count', ascending=False).show(10)

+--------------+------+
community_area| count|
+--------------+------+
 25|408898|
 8|224209|
 43|210864|
 23|203719|
 28|191694|
 24|190311|
 67|188672|
 29|187692|
 71|183030|
 49|171945|
+--------------+------+
only showing top 10 rows

In [58]:

## Is it possible to see how the top crime types occur in the top crime-affected areas?
top_crime_types = df_dates.select('primary_type').groupBy('primary_type').count().rdd.map(lambda row: row.asDict()).takeOrdered(10, key=lambda l: 1/l['count'])
top_busy_areas =  df_dates_community_areas.rdd.map(lambda row: row.asDict()).takeOrdered(10, key=lambda l: 1/l['count'])

In [59]:
top_crime_types_lst = [dc['primary_type'] for dc in top_crime_types]
top_busy_areas_lst = [dc['community_area'] for dc in top_busy_areas]

In [60]:
top_crime_types_lst


Out[61]: ['THEFT',
 'BATTERY',
 'CRIMINAL DAMAGE',
 'NARCOTICS',
 'ASSAULT',
 'OTHER OFFENSE',
 'BURGLARY',
 'MOTOR VEHICLE THEFT',
 'DECEPTIVE PRACTICE',
 'ROBBERY']

In [61]:
top_busy_areas_lst

Out[62]: [25, 8, 43, 23, 28, 24, 67, 29, 71, 49]

In [62]:
q1 = "instr('" + ' '.join(top_busy_areas_lst) + "', community_area) > 0"
q2 = "instr('" + ' '.join(top_crime_types_lst) + "', primary_type) > 0"
print(q1)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-4442033637449669> in <module> 
 ----> 1 q1 = "instr('" + ' ' . join ( top_busy_areas_lst ) + "', community_area) > 0" 
 2 q2 = "instr('" + ' ' . join ( top_crime_types_lst ) + "', primary_type) > 0" 
 3 print ( q1 ) 

 TypeError : sequence item 0: expected str instance, int found

In [63]:

## Construct a data frame filtered on these top community areas and top crime types:
df_dates_tops = df_dates.filter(q1).filter(q2)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4442033637449670> in <module> 
 1 
 2 ## Construct a data frame filtered on these top community areas and top crime types: 
 ----> 3 df_dates_tops = df_dates . filter ( q1 ) . filter ( q2 ) 

 NameError : name 'q1' is not defined

In [64]:
df_dates_tops.count()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4442033637449671> in <module> 
 ----> 1 df_dates_tops . count ( ) 

 NameError : name 'df_dates_tops' is not defined

In [65]:
tops_of_tops = df_dates_tops.groupBy(['primary_type', 'community_area']).count().orderBy(['primary_type', 'count', 'community_area'], ascending=[True, False, True]).cache()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4442033637449672> in <module> 
 ----> 1 tops_of_tops = df_dates_tops . groupBy ( [ 'primary_type' , 'community_area' ] ) . count ( ) . orderBy ( [ 'primary_type' , 'count' , 'community_area' ] , ascending = [ True , False , True ] ) . cache ( ) 

 NameError : name 'df_dates_tops' is not defined

In [66]:
tops_of_tops.show(20)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4442033637449673> in <module> 
 ----> 1 tops_of_tops . show ( 20 ) 

 NameError : name 'tops_of_tops' is not defined

In [67]:
area_names = """
01	Rogers Park	
40	Washington Park
02	West Ridge	
41	Hyde Park
03	Uptown	
42	Woodlawn
04	Lincoln Square	
43	South Shore
05	North Center	
44	Chatham
06	Lakeview	
45	Avalon Park
07	Lincoln Park	
46	South Chicago
08	Near North Side	
47	Burnside
09	Edison Park	
48	Calumet Heights
10	Norwood Park	
49	Roseland
11	Jefferson Park	
50	Pullman
12	Forest Glen	
51	South Deering
13	North Park	
52	East Side
14	Albany Park	
53	West Pullman
15	Portage Park	
54	Riverdale
16	Irving Park	
55	Hegewisch
17	Dunning	
56	Garfield Ridge
18	Montclare	
57	Archer Heights
19	Belmont Cragin	
58	Brighton Park
20	Hermosa	
59	McKinley Park
21	Avondale	
60	Bridgeport
22	Logan Square	
61	New City
23	Humboldt Park	
62	West Elsdon
24	West Town	
63	Gage Park
25	Austin	
64	Clearing
26	West Garfield Park 	
65	West Lawn
27	East Garfield Park	
66	Chicago Lawn
28	Near West Side	
67	West Englewood
29	North Lawndale	
68	Englewood
30	South Lawndale	
69	Greater Grand Crossing
31	Lower West Side	
70	Ashburn
32	Loop	
71	Auburn Gresham	
33	Near South Side	
72	Beverly
34	Armour Square	
73	Washington Heights
35	Douglas	
74	Mount Greenwood
36	Oakland	
75	Morgan Park
37	Fuller Park	
76	O'Hare
38	Grand Boulevard	
77	Edgewater
39	Kenwood	
"""

In [68]:
code_pairs = [[float(p[0]), p[1]] for p in [pair.strip().split('\t') for pair in area_names.strip().split('\n')]]

In [69]:
code_pairs[:5]

Out[70]: [[1.0, 'Rogers Park'],
 [40.0, 'Washington Park'],
 [2.0, 'West Ridge'],
 [41.0, 'Hyde Park'],
 [3.0, 'Uptown']]

In [70]:
community_area_counts = pd.DataFrame(df_dates_community_areas.rdd.map(lambda row: row.asDict()).collect())

In [71]:
# Create a dictionary of area code to names
area_name_dic = {float(k[0]):k[1] for k in code_pairs}

In [72]:
community_area_counts['community_area_name'] = community_area_counts['community_area'].apply(lambda area: area_name_dic.get(float(area),  'unknown_%s'%area))

In [73]:

community_area_counts = community_area_counts.sort_values(by='count')
community_area_counts.head(5)

,community_area,count,community_area_name
74,0,93,unknown_0
32,9,6243,Edison Park
11,47,9648,Burnside
9,12,11566,Forest Glen
37,55,13965,Hegewisch


In [74]:
selected_features = [
 'location_description',
 'arrest',
 'domestic',
 'beat',
 'district',
 'ward',
 'community_area',
 'fbi_code',
 'hour',
 'week_day',
 'year_month',
 'month_day',
 'date_number']

In [75]:
#Let's see the schema of these selected features:
features_df = df_dates.select(selected_features)
features_df.printSchema()

root
-- location_description: string (nullable = true)
-- arrest: boolean (nullable = true)
-- domestic: boolean (nullable = true)
-- beat: integer (nullable = true)
-- district: integer (nullable = true)
-- ward: integer (nullable = true)
-- community_area: integer (nullable = true)
-- fbi_code: string (nullable = true)
-- hour: integer (nullable = true)
-- week_day: integer (nullable = true)
-- year_month: integer (nullable = true)
-- month_day: integer (nullable = true)
-- date_number: integer (nullable = true)

In [76]:
feature_level_count_dic = []

for feature in selected_features:
    print('Analysing %s' % feature)
    levels_list_df = features_df.select(feature).distinct()
    feature_level_count_dic.append({'feature': feature, 'level_count': levels_list_df.count()})

Analysing location_description
Analysing arrest
Analysing domestic
Analysing beat
Analysing district
Analysing ward
Analysing community_area
Analysing fbi_code
Analysing hour
Analysing week_day
Analysing year_month
Analysing month_day
Analysing date_number

In [77]:
pd.DataFrame(feature_level_count_dic).sort_values(by='level_count', ascending=False)

,feature,level_count
12,date_number,6974
3,beat,304
0,location_description,181
6,community_area,79
5,ward,51
11,month_day,31
7,fbi_code,26
4,district,25
8,hour,24
10,year_month,12


In [78]:
####Preparing a model###
from pyspark.ml.feature import StringIndexer, VectorAssembler,CountVectorizer,Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
df_dates_features = df_dates.na.drop(subset=selected_features)

In [79]:
for feature in feature_level_count_dic:
    indexer = StringIndexer(inputCol=feature['feature'], outputCol='%s_indexed' % feature['feature'])
    print('Fitting fture "%s"' % feature['feature'])
    model = indexer.fit(df_dates_features)
    print('Transforming "%s"' % feature['feature'])
    df_dates_features = model.transform(df_dates_features)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o3485.fit.
: java.lang.IllegalArgumentException: requirement failed: The input column arrest must be either string type or numeric type, but got BooleanType.
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.feature.StringIndexerBase$class.validateAndTransformSchema(StringIndexer.scala:87)
	at org.apache.spark.ml.feature.StringIndexer.validateAndTransformSchema(StringIndexer.scala:110)
	at org.apache.spark.ml.feature.StringIndexer.transformSchema(StringIndexer.scala:153)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.feature.StringIndexer.fit(StringIndexer.scala:136)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-4442033637449686> in <module> 
 2 indexer = StringIndexer ( inputCol = feature [ 'feature' ] , outputCol = '%s_indexed' % feature [ 'feature' ] ) 
 3 print ( 'Fitting fture "%s"' % feature [ 'feature' ] ) 
 ----> 4 model = indexer . fit ( df_dates_features ) 
 5 print ( 'Transforming "%s"' % feature [ 'feature' ] ) 
 6 df_dates_features = model . transform ( df_dates_features ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 77 raise QueryExecutionException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 78 if s . startswith ( 'java.lang.IllegalArgumentException: ' ) : 
 ---> 79 raise IllegalArgumentException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 80 raise 
 81 return deco

 IllegalArgumentException : 'requirement failed: The input column arrest must be either string type or numeric type, but got Bo

In [80]:
## String-index the response variable:
response_indexer = StringIndexer(inputCol='primary_type', outputCol='primary_type_indexed')
response_model = response_indexer.fit(df_dates_features)
df_dates_features = response_model.transform(df_dates_features)

In [81]:

#What does it look like now...
df_dates_features.show(1)

+--------+-----------+--------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+-------------------+----------+----+--------+----------+---------+-----------+----------------------------+--------------------+
 id|case_number| date| block|iucr| primary_type| description|location_description|arrest|domestic|beat|district|ward|community_area|fbi_code|x_coordinate|y_coordinate|year| updated_on|latitude|longitude|location| date_time| month|hour|week_day|year_month|month_day|date_number|location_description_indexed|primary_type_indexed|
+--------+-----------+--------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+-------------------+----------+----+--------+----------+---------+-----------+----------------------------+--------------------+
11034701| JA366925|01/01/2001 11:00:...|016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...| RESIDENCE| false| false| 412| 4| 8| 45| 11| null| null|2001|08/05/2017 03:50:...| null| null| null|2001-01-01 11:00:00|2001-01-01| 11| 2| 1| 1| 0| 1.0| 8.0|
+--------+-----------+--------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+-------------------+----------+----+--------+----------+---------+-----------+----------------------------+--------------------+
only showing top 1 row

In [82]:
indexed_features = ['%s_indexed' % fc['feature'] for fc in feature_level_count_dic]
indexed_features

Out[83]: ['location_description_indexed',
 'arrest_indexed',
 'domestic_indexed',
 'beat_indexed',
 'district_indexed',
 'ward_indexed',
 'community_area_indexed',
 'fbi_code_indexed',
 'hour_indexed',
 'week_day_indexed',
 'year_month_indexed',
 'month_day_indexed',
 'date_number_indexed']

In [83]:
assembler = VectorAssembler(inputCols=indexed_features, outputCol='features')
vectorized_df_dates = assembler.transform(df_dates_features)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o3675.transform.
: java.lang.IllegalArgumentException: Field "arrest_indexed" does not exist.
Available fields: id, case_number, date, block, iucr, primary_type, description, location_description, arrest, domestic, beat, district, ward, community_area, fbi_code, x_coordinate, y_coordinate, year, updated_on, latitude, longitude, location, date_time, month, hour, week_day, year_month, month_day, date_number, location_description_indexed, primary_type_indexed
	at org.apache.spark.sql.types.StructType$$anonfun$apply$1.apply(StructType.scala:274)
	at org.apache.spark.sql.types.StructType$$anonfun$apply$1.apply(StructType.scala:274)
	at scala.collection.MapLike$class.getOrElse(MapLike.scala:128)
	at scala.collection.AbstractMap.getOrElse(Map.scala:59)
	at org.apache.spark.sql.types.StructType.apply(StructType.scala:273)
	at org.apache.spark.ml.feature.VectorAssembler$$anonfun$6.apply(VectorAssembler.scala:164)
	at org.apache.spark.ml.feature.VectorAssembler$$anonfun$6.apply(VectorAssembler.scala:163)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.TraversableLike$$anonfun$flatMap$1.apply(TraversableLike.scala:241)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at scala.collection.TraversableLike$class.flatMap(TraversableLike.scala:241)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:186)
	at org.apache.spark.ml.feature.VectorAssembler.transformSchema(VectorAssembler.scala:163)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.feature.VectorAssembler.transform(VectorAssembler.scala:88)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-4442033637449690> in <module> 
 1 assembler = VectorAssembler ( inputCols = indexed_features , outputCol = 'features' ) 
 ----> 2 vectorized_df_dates = assembler . transform ( df_dates_features ) 

 /databricks/spark/python/pyspark/ml/base.py in transform (self, dataset, params) 
 171 return self . copy ( params ) . _transform ( dataset ) 
 172 else : 
 --> 173 return self . _transform ( dataset ) 
 174 else : 
 175 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _transform (self, dataset) 
 310 def _transform ( self , dataset ) : 
 311 self . _transfer_params_to_java ( ) 
 --> 312 return DataFrame ( self . _java_obj . transform ( dataset . _jdf ) , dataset . sql_ctx ) 
 313 
 314 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j

In [84]:

vectorized_df_dates.select('features').take(0)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4442033637449691> in <module> 
 1 
 ----> 2 vectorized_df_dates . select ( 'features' ) . take ( 0 ) 

 NameError : name 'vectorized_df_dates' is not defined

In [85]:
train, test = vectorized_df_dates.randomSplit([0.6, 0.4])

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4442033637449692> in <module> 
 ----> 1 train , test = vectorized_df_dates . randomSplit ( [ 0.6 , 0.4 ] ) 

 NameError : name 'vectorized_df_dates' is not defined